# Cardiovascular disease Prediction

Cardiovascular diseases are the No.1 reason for all deaths in the world [[WHO](https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds) )] and many are preventable.

There are many subgroups of different kind of diseases centred around the heart and the blood vessels:

- coronary heart disease – disease of the blood vessels supplying the heart muscle
- cerebrovascular disease – disease of the blood vessels supplying the brain
- peripheral arterial disease – disease of blood vessels supplying the arms and legs
- rheumatic heart disease – damage to the heart muscle and heart valves from rheumatic fever, caused by streptococcal bacteria
- congenital heart disease – malformations of heart structure existing at birth
- deep vein thrombosis and pulmonary embolism – blood clots in the leg veins, which can dislodge and move to the heart and lungs.

The typical risk factors of Cardiovascular diseases are:

- Behavioral:
    - diet,
    - physical inactivity,
    - tobacco use and
    - harmful use of alcohol.

- Medical Values:
    - raised blood pressure,
    - raised blood glucose,
    - raised blood lipids and
    - overweight & obesity


The risk factors mentioned above are quite numerous, and a doctor is only so good at asking questions and data control.
Our motivation for this project is to help doctors and patients by providing a tool that is able to make a prediction if one it subjected to a cardiovascular disease.
This prediction is based on objective, measured and subjective information where most details can be easily obtained by the patients themselves or simple measurements.
The Machine Learning model might therefore be a useful tool to bring attention to early stages and to minimize examination mistakes by providing a second opinion.

## Dataset

To make this prediction possible we found a dataset  with containing said risk factors as features, and the label based on the Cardiovascular disease status.

This dataset is taken from Kaggle.com and can be found [here](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset).

The dataset has data recordings of 70 000 patients including 11 different Features and 1 label. There are 3 types of input features:

* Objective:    factual information
* Examination:  results of medical examination
* Subjective:   information given by the patient

| [index] id| [0] age| [1] gender| [2] height| [3] weight|
| ---| ---| ---| ---| ---|
| int| int| 1 or 2 | int| float|
| -| days| categorical code (2=men)| cm| kg |
| -| Objective| Objective| Objective| Objective |

| [4] ap_hi| [5] ap_lo| [6] cholesterol| [7] gluc|
| ---| ---| ---| ---|
| int| int| 1, 2, 3 | 1, 2, 3 |
| -| -| normal, above normal, well above normal| normal, above normal, well above normal|
| Examination| Examination| Examination| Examination|

>Note: ap_hi = Systolic blood pressure, ap_lo = Diastolic blood pressure, gluc = Glucose

| [8] smoke| [9] alco| [10] active| [11] cardio|
| ---| ---| ---| ---|
| binary| binary| binary| binary |
| -| -| -| categorical code|
| Subjective| Subjective| Subjective| Target|

>Note: alco = Alcohol intake



## EDA - Data Correlation

A good start for choosing the correct prediction algorithm is an exploratory data analysis (EDA).
This analysis looks at the data and shows potential correlations or problems.

More information and details can be found in this [report](../docs/CardioVascular-Diseas-Prediction.html).
This report was compiled using the [pandas-profiling tool](https://github.com/pandas-profiling/pandas-profiling).

 - Our results show that some data is corrupt and needs to be fixed (e.g. implausible blood pressure values).
 - The data is not linear separable, so we definitely need a good feature function or kernel.
 - The result of the prediction should be a label and not a value, so we want a classifier.


Based on the these results a good choice for a prediction algorithm in our project is the **Kernel Logistic Regression**.


Based on the WHO's [cardiovascular risk charts](https://www.who.int/news/item/02-09-2019-who-updates-cardiovascular-risk-charts)
being male/elderly/a smoker or having diabetes/high cholesterol levels are the most prominent risk factors for having a cardiovascular disease.


To get a better understanding of our data set and the relation between the variables, we compute the correlation matrix.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


%matplotlib inline
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
plot_df = pd.read_csv('../resources/cardio_train.csv', sep=';', index_col='id')#.sample(1000, random_state=42)

plt.figure(figsize=(14, 8))
sns.heatmap(plot_df.corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')
plt.title('Correlation Matrix', fontdict={'fontsize':14}, pad=12)

### Data Abnormalities

By looking into the last column, we can see the variables of our dataset which correlate most to a cardiovascular disease.
In our case the biggest influences are the features `age`, `cholesterol`, `weight`, `gulcose` and blood pressure (`ap_lo`/`ap_hi`) (with decreasing significance).

This aligns mostly with the medical values mentioned by the WHO.

Contrary to the aforementioned WHO's cardiovascular risk charts, the `gender` of a person is of minor importance.

Surprisingly, `smoking` and high `alcohol` intake seem to lessen the risk of cardiovascular disease.
Perhaps these features may have gotten mixed up during data collection, but as this is a kaggle-dataset with no reference to the original source,
we cannot know for sure.

We plot the features `age` and `weight` in order to visualize the trend in the data.

In [ ]:
# take age & round days to years
age = plot_df['age']
age_divider = 1.0/365.0
age = age * age_divider
#
# create age data in correlation with cardio
age_data = pd.concat([age, plot_df['weight'], plot_df['cardio']], axis=1, join='inner')

plt.figure(figsize=(10, 6))
sns.scatterplot(x='age', y='weight', data=age_data, hue='cardio', palette="seismic")
# TODO label into ['cardio true', 'cardio false']
plt.xlabel('age in years')
plt.ylabel('weight in kg')


As expected, there tend to be more healthy cases in the lower left part of the scatter plot (low age/ low weight) than in the upper right (high age/ high weight).
Nevertheless, it should also be noted that there are many outliers, as well as some values that don't make any sense (e.g. an adult person with only 10 kg).

It may be advantageous to exclude these outliers before training our model.

The plot also shows that our dataset seems to have a weird peculiarity in the feature `age`.
Instead of a smooth distribution over the years, some clusters can be observed.
While we do convert the age (given in days) into years, we keep our value as a float and do not round the value.
We assume that some kind of limitation in the creation process is the reason for these jumps in the data. Our dataset might just be a subset of an even larger dataset as it mostly includes patients in the age group 39-66 years. 
This peculiarity does cause some concern for trust into data, but for our project it is important that no unexpected trends emerge, and our dataset is still a representation of the real world.

Luckily this is the case: when compared with the WHO's risk tables the trend of "higher age, more of cases of cardio diseases" is still valid.

### Duplicate Entries

Our dataset is clearly not very clean.
If it also contains duplicate rows, they may end up in both the training set and the test set, which might be a problem
for estimating the generalization error if they are the result of the poor data collection process.
However, with a dataset consisting of 70000 entries some duplicates are to be expected.

There appears to be only 41 entries with the exact same feature values - with only 24 of these also matching the same label.
We conclude that there is no need for deleting these duplicate rows, because they probably represent real world data.

In [ ]:
features = plot_df.columns[:-1]

duplicates_features = plot_df[plot_df.duplicated(features)]
duplicates = plot_df[plot_df.duplicated()]

print(f"Our dataset contains {len(duplicates_features)} duplicates (varying labels)")
print(f"Our dataset contains {len(duplicates)} duplicates (matching labels)")

In [ ]:
# Clean up Jupyter Notebook for better performance
%reset -f

# Kernel Logistic Regression

#### Formulary
Features: $x$

Labels: $y$

Hypothesis Function: $h(x)$

Loss Function: $l(h(x), y)$

Regularization Term: $\Omega(w)$

Objective Function: $ J(w) = \frac{1}{m} \sum_{i=1}^{m} l(h(x_i), y_i) + \Omega(w)$


### Imports

In [ ]:
from typing import Tuple

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

%load_ext autotime

### Load Data

In [ ]:
df = pd.read_csv('../resources/cardio_train.csv', sep=';', index_col='id')

### Removing corrupted entries

As we saw in the explorative data analysis, our dataset includes corrupted values (e.g. blood pressures above 20000) that need to be expelled before we can make a solid prediction. 

Therefore we define reasonable value ranges for the systolic/diastolic blood pressures, weight and height features in order to account for any errors during the data collection process.

In [ ]:
org_count = len(df)

df = df[(50 <= df['ap_lo']) & (df['ap_lo'] <= 150)]
df = df[(100 <= df['ap_hi']) & (df['ap_hi'] <= 200)]

df = df[(25 <= df['weight']) & (df['weight'] <= 400)]
df = df[(100 <= df['height']) & (df['height'] <= 210)]

new_count = org_count - len(df)
print(f"{new_count} entries have been excluded due to implausible feature values.")

del new_count, org_count

### Feature Scaling

The information provided by the dataset depends on the category. Some are binary values like `gender`, others are categorical like `cholesterol` or numerical like `weight` and `height`.
We need to normalize these data points to a similar scale. This process is called feature scaling.

Feature Scaling is necessary because if the range of raw data varies widely, it can be the case that the objective function of some machine learning algorithms will not work properly.
This is also the case for the Kernel Feature Function.
The main reason for this is that the Kernel Logistic Regression Algorithm calculates the Squared/Euclidean distance between the Feature Points.
If one Feature has a broad value range the distance is determined and influenced mostly by this particular feature.

Since the Cardiovascular Disease Dataset has a broad value range for example for the Feature `age` (given in days), we standardize our data such that all features have a mean of zero and a standard deviation of 1.
We use the standardization formula:

$\tilde{x_i} = \frac{x_i - \mu}{\sigma}$

In [ ]:
import statistics

def standardize(feature):
    return (feature-statistics.mean(feature)) / statistics.stdev(feature)

df_standardized = pd.DataFrame({})

# numerical values
df_standardized['age'] = standardize(df['age'])
df_standardized['height'] = standardize(df['height'])
df_standardized['weight'] = standardize(df['weight'])
df_standardized['ap_hi'] = standardize(df['ap_hi'])
df_standardized['ap_lo'] = standardize(df['ap_lo'])

# binary/categorical values
df_standardized['gender'] = df['gender'].apply(lambda t: -1 if t==1 else 1).values
df_standardized['smoke'] = df['smoke'].apply(lambda t: -1 if t==0 else 1).values
df_standardized['alco'] = df['alco'].apply(lambda t: -1 if t==0 else 1).values
df_standardized['active'] = df['active'].apply(lambda t: -1 if t==1 else 1).values
df_standardized['cholesterol'] = df['cholesterol'].apply(lambda t: -1 if t==1 else (0 if t==2 else 1)).values
df_standardized['gluc'] = df['gluc'].apply(lambda t: -1 if t==1 else (0 if t==2 else 1)).values

df_standardized['cardio'] = df['cardio'].apply(lambda t: 1 if t==1 else -1).values


del df

### Kernel / Feature function
For the best result we want to include many of our important risk factors into our feature function.

Based on our EDA we can say that our data is - as it is - not linear separable. There might be a case where the data is linear separable if aligned properly, but instead of finding the best feature function by introducing extra dimensions, we kernelize our feature function.

$h(x) = w^T * \Phi(x_i)= w^T*K(x,z) = w^T * (x^T*z+1)^d $

## Implementing the objective function + helper functions

#### Squared Exponential Kernel $k(x,z)$
$k(x,z) = exp(− x^Tx−2x^Tz+z^Tz/ 2σ^2) = exp(sqdist(x,z)/2σ^2)$

#### Hypothesis Function $h(x)$
$h_\alpha(x) = \alpha K = \sum_{j=1}^{m} \alpha_j k(x_j,x)$

#### Loss Function $l(h(x),y)$

logistic loss:

$l_{logistic}(h_\alpha(x), y) = log(1 + e^{−y·h(x)})= log(1 + exp(−y · h_\alpha(x)))$

#### Regularization Term: $\Omega(w)$

$\Omega(\alpha) = \lambda l_2 = \lambda\alpha^{\intercal}K\alpha$


#### Objective Function J

kernelized logistic regression:

$
J(\alpha)
= \frac{1}{m} \sum_{i=1}^{m} l(h(x_i), y_i) + \Omega(\alpha)
= \frac{1}{m} \sum_{i=1}^m  \log \big(1 + \exp\big(-y_i \cdot \sum_{j=1}^{m} \alpha_j k(x_j,x_i)\big) \big) + \lambda \alpha^{\intercal}K\alpha
$

In [ ]:
def sqdist(X, Z):
    p1 = np.sum(X**2, axis=1)[:, np.newaxis]
    p2 = np.sum(Z**2, axis=1)
    p3 = -2 * np.dot(X, Z.T)
    return p1+p2+p3

def sq_exp(X, Z, sigma):
    return np.exp(-sqdist(X, Z)/(2*sigma**2) )


def J(α, X, y, sigma, lam):
    K = sq_exp(X, X, sigma)
    m = X.shape[0]
    total_loss = 0
    regularization = lam * np.dot(np.dot(np.transpose(α), K), α)

    for i in range(m):
        prediction = α @ K[i]
        logistic_loss = np.log(1 + np.exp(-y[i] * prediction))
        total_loss += logistic_loss

    mean_loss = total_loss / m  + regularization
    return mean_loss

Implement the gradient of the regularized kernlized logistic regression objective.

In [ ]:
def dJ(α, X, y, sigma, lam):
    K = sq_exp(X, X, sigma)
    m = X.shape[0]
    gradient = 0
    regularization = 2*lam * np.dot( K, α)

    for i in range(m):
        prediction = α @ K[i]

        numerator = -y[i] * K[i]
        denominator = 1 + np.exp(y[i] * prediction)
        gradient += numerator / denominator

    mean_gradient = gradient / m + regularization
    return mean_gradient

## Training our model

In [ ]:
from scipy.optimize import minimize

def kernel_lr(X, y, sigma, lam):
    # implementation of kernel ridge regression using the scipy optimizer gradient descent
    α = np.zeros(X.shape[0],)
    α = minimize(J, α, args=(X, y, sigma, lam), jac=dJ, method='CG').x
    h = lambda Z: np.dot(α, sq_exp(X, Z, sigma))
    return h

### Split data in Train / Validation / Test

In order to determine the quality of our model, we have to estimate its generalization error (on new data).
Therefore, it is common to split the dataset into a training set and a test set which will be exclusively used for training and testing respectively. 

However, the Squared Exponention Kernel depends on the parameter $\sigma$ and the-$\lambda$ parameter determines how much we regularize our model. First, we need to optimize these parameters by training multiple models with varying ($\sigma$, $\lambda$)-pairs. The resulting models need to be compared, but we cannot use our test set for the gerneralization error, because we would be "cherry-picking" the model that best works for the test set instead of real world data.

In order to counteract this problem, we use cross-validation for estimating the gerneralization error. Our training set needs to be split into $k$ folds. We than train $k$ models on $k-1$ training folds und use the last fold for validating our models (each model with a different validation fold). The average over all validation errors serves as a guide for choosing the best parameters $\sigma$ and $\lambda$. 

We will train one last model with optimal parameters on the entire training set, and only than use the test set for estimating the gerneralization error.

In [ ]:
def train_test_split(data, train_sample_count, shuffle=True):
    mask = np.full(data.shape[0], False)
    mask[:train_sample_count] = True

    if shuffle:
        np.random.seed(42)
        np.random.shuffle(mask)

    train_data = data[mask]
    test_data = data[~mask]

    return train_data, test_data

In [ ]:
def cross_val(data, k=10):
    assert k >= 2
    datasets = []

    if data.shape[0] % k != 0:
        print("warning: this dataset contains {} entries and cannot be equally divided into {} chunks for cross-validation.".format(data.shape[0], k))
        print("Prutruding rows will be dropped.")
        data = data[ : (data.shape[0] // k) * k]

    for i in range(k):
        data_chunks = np.split(data, k)

        val_data = data_chunks.pop(i)
        train_data = np.concatenate(data_chunks)
        datasets.append((train_data, val_data))

    return datasets

It is common to use 70% of the data for training purposes and 30% for testing. However, training the Kernel Logistic Regression Algorithm for lots of datapoints is computationally expensive. Unfortunately we already reach our computational limit (in a reasonable amount of time) with ~1000 datapoints. We have an obundantly large dataset and predicting on an already trained model is quite fast. Therefore, our test set will be unusually large compared to our training set.

The computational limitations also force us to use a small number of folds for cross-validation. We will only use 2-fold-cross-validation.

In [ ]:
cardio_data = df_standardized.to_numpy()

training_samples = 500
fold_count = 2

train_data, test_data = train_test_split(cardio_data, training_samples, shuffle=True)
cross_val_datasets = cross_val(train_data, k=fold_count)

print(f"training set size: {train_data.shape[0]} ")
print(f"test set size: {test_data.shape[0]} ")

del cardio_data

### Choose Features

Instead of using all features available we choose to only take these features that align with our theoretical background.
This means we look at the correlation matrix from the EDA, which results in following claims:

#### Good Features:
`age`, `weight`, `cholesterol`, `gluc`, `ap_lo` and `ap_hi` support our hypothesis.

`active` ?????
TODO: what about being active thats a feature that should be useful?

As these features a helpful for our model we include these as our features.

#### Neutral Features:
`gender` doesn't show high enough correlation although it would support our hypothesis.

`height` should have no influence on having a cardiovascular disease.

For faster results while computing the prediction we leave these features out.

#### Bad Features:
`alco` and `smoking` normally show a strong correlation towards having a cardiovascular disease.
A minor opposite correlation can be found in out dataset.

For a better correlation we leave these features out.

In [ ]:
def get_labels_and_features(dataset:np.ndarray)->Tuple[np.ndarray, np.ndarray]:
    """Return labels and features from a given dataset.
    :return: labels, features
    """
    col = {
        'age': 0, 'height': 1 , 'weight': 2, 'ap_hi': 3, 'ap_lo': 4, 'gender': 5, 'smoke': 6, 
        'alco': 7, 'active': 8, 'cholesterol': 9, 'gluc': 10, 'cardio': 11
           }
    feature_list = [
        col['age'], col['weight'], col['cholesterol'], col['gluc'], col['ap_lo'], col['ap_hi'], col['active'], col['gender']
        ]

    labels = dataset[:, col['cardio']]
    features = dataset[:, feature_list]
    return labels, features

### Parameter Optimization

TODO Vinc: Beschreiben unserer measures against over and underfitting

In [ ]:
def score(h, X, y):
    #TODO replace score with precision/recall score from metrics
    predictions = h(X)

    score = (predictions*y >= 0).astype(int)
    return score.sum()/score.shape[0]


sigmas=[0.5, 1., 2., 4., 8.]
lambdas=[1.]

for lam in lambdas:
    for sigma in sigmas:
        train_scores = []
        val_scores = []
        for train_set, val_set in cross_val_datasets:
            y_train, X_train = get_labels_and_features(train_set)
            y_val, X_val = get_labels_and_features(val_set)

            h = kernel_lr(X_train, y_train, sigma=sigma, lam=lam)

            train_scores.append(score(h, X_train, y_train))
            val_scores.append(score(h, X_val, y_val))
        
        print(f'Average model accuracy for sigma={sigma}, lambda={lam}')
        print(f'train: {sum(train_scores)/len(train_scores)}')
        print(f'val: {sum(val_scores)/len(val_scores)}\n')


# TODO Jan: show generalization error in plot, or plots for lambda / simga changes
# compare: https://en.wikipedia.org/wiki/Receiver_operating_characteristic

## Metrics

For a final evaluation of our model we calculate use common metrics for comparability.

Our metrics are based around the confusion matrix.

True positive = $TP$

True negative = $TN$

False positive = $FP$

False negative = $FN$

- Accuracy: the proportion of correct predictions among the total number of predictions made.

    $Accuracy = (TP + TN) / (TP + TN + FP + FN) $

    So how accurate is our prediction of datapoints. Note: an overfitted model has a high accuracy.

- Precision: the proportion of the true predictions among all the positive predictions made.

    $Precision = TP / (TP + FP)$

    So how serious do we have to take a positive result. Note: when this score is low, we have lots of false alarms.

- Recall: the proportion of the true positive predictions among the total amount of relevant samples in the dataset.

    $Recall = TP / (TP + FN)$

    So how good is our model in detecting the disease. Note: when this score is low, we missed lots of cases where patients have the disease.

- F1: the harmonic mean of the precision and recall.

    $F1 = 2 * (precision * recall) / (precision + recall)$

    As neither accuracy, precision and recall can judge the model's performance on their own we use the F1 score as a final comparsion value.
    This works because the harmonic mean puts the focus on the small values.
    So when either precision **or** recall is performing badly it reflects in the F1 score.


In [ ]:
# train hypothesis function on entire training set (including validation set) on best parameters
sigma = 0.9
lam = 1e-4
y_train, X_train = get_labels_and_features(train_data)
y_test, X_test = get_labels_and_features(test_data)
h = kernel_lr(X_train, y_train, sigma=sigma, lam=lam)

In [ ]:
# calculate metrics
X, y =  X_test, y_test

predictions = h(X)
matching_score = (predictions * y >= 0).astype(int)

# true disease, false alarm, no disease, missed disease
tp, fp, tn, fn = [], [], [], []
for index, score in enumerate(predictions):
    label = y[index]
    if score > 0 and label > 0:
        tp.append(score)
    elif score > 0 and label < 0:
        fp.append(score)
    elif score < 0 and label < 0:
        tn.append(score)
    elif score < 0 and label > 0:
        fn.append(score)


accuracy = matching_score.sum() / matching_score.shape[0]
precision = len(tp)/ (len(tp)+ len(fp))
recall = len(tp) / (len(tp) + len(fn))
f_1 = 2 * ( (precision * recall) / (precision + recall) )


print('\t---- Metrics ----\n'
    f'Accuracy:\t {accuracy}\n'
    f'Precision:\t {precision}\n'
    f'Recall:\t\t {recall}\n'
    f'F1:\t\t {f_1}\n')


print('\t---- Confusion Matrix ----')
plot_data = [[len(tn), len(fp)],
             [len(fn), len(tp)]]
sns.heatmap(plot_data, annot=True)
plt.xlabel('prediction')
plt.ylabel('actual')
plt.show()


print(f'We predicted correctly that {len(tn)} patients have no disease.\n'
    f'We predicted correctly that {len(tp)} patients have a cardio disease.\n'
    f'But we missed {len(fn)} patients and gave {len(fp)} false alarms.\n')

ToDo: Generalisierungsfehler schätzen mit Test-Datensatz

true positive rate (tpr) & true negative rate (tnr) ?
tpr = recall
tnr =  len(tn)/(len(tn)+len(fp))


## Further Cases

Cardiovascular disease Prediction

TODO Vinc: Ausblick -> z.B. andere Lern algorithm, ensemble Methoden wie adaboost, ...
Ausblick wenn mehr Zeit - do we want to show any differences compared to decision trees , SVM's, etc...

TODO: compare our results with others on kaggle

